# Youtube Data API

In [ ]:
!pip install google-api-python-client

In [7]:
import json
from googleapiclient.discovery import build

In [8]:
DEVELOPER_KEY = json.load(open('./keys/youtube_key.json'))['api_key']
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'

youtube = build(
    YOUTUBE_API_SERVICE_NAME,
    YOUTUBE_API_VERSION,
    developerKey=DEVELOPER_KEY
)


In [9]:
# Get Channel object

# 1. First find Channel ID by Channel Name

request = youtube.search().list(
        part="snippet",
        maxResults=5,
        q="Barbascura eXtra",
        type="channel"
    )

response = request.execute()

channelId = response['items'][0]['id']['channelId']

channelId

'UCHi6Q3Z-5oJUC691WLlSntA'

In [10]:
# 2. Then fetch the information we need about it

request = youtube.channels().list(
    part="snippet,contentDetails",
    id= channelId
    )
response = request.execute()
uploaded_playlist = response['items'][0]['contentDetails']['relatedPlaylists']['uploads']
uploaded_playlist

'UUHi6Q3Z-5oJUC691WLlSntA'

In [11]:
channel_videos : list = list()

page_token = None

while True:
    request = youtube.playlistItems().list(
            part="snippet,contentDetails",
            maxResults=50,
            playlistId=uploaded_playlist,
            pageToken = page_token
    )

    response = request.execute()

    channel_videos.extend(
        v['contentDetails']['videoId']
        for v in response['items']
    )

    if response.get('nextPageToken', None) is None:
        break

    page_token = response['nextPageToken']

len(channel_videos)

379

In [13]:
import time

# Get Comments for Every Video
commentators : dict[str, set[str]] = {}

for video_id in channel_videos[:10]:

    page_token = None

    video_commentators : set = set()

    while True:
        request = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            maxResults=100,
            pageToken = page_token
        )

        response = request.execute()

        video_commentators.update(
            ct['snippet']['topLevelComment']['snippet']['authorDisplayName']
            for ct in response['items']
        )

        if response.get('nextPageToken', None) is None:
            break

        page_token = response['nextPageToken']

    time.sleep(0.1)
    commentators[video_id] = video_commentators


In [14]:
len(commentators)

10

In [55]:
# Now use networkx to create a Network:
#   * Nodes are the Videos
#   * The Edge (a, b) is present if a and b share some commentators

In [15]:
import networkx as nx

nodes_list = list(commentators.keys())
edges_list = []

for i in range(0, len(nodes_list)):
    video1 = nodes_list[i]
    for j in range(i+1, len(nodes_list)):
        video2 = nodes_list[j]

        common_commentators = commentators[video1] & commentators[video2]

        if len(common_commentators) > 0:
            edges_list.append((video1, video2, len(common_commentators)))


edges_list


[('wUArIUoJxkA', 'oHg8ETcoXIA', 11),
 ('wUArIUoJxkA', 'YydAr-kkobs', 15),
 ('wUArIUoJxkA', 'xpU3TBmoZB4', 16),
 ('wUArIUoJxkA', 'k7HHjV_IzPs', 6),
 ('wUArIUoJxkA', '_E33gRrWUHM', 6),
 ('wUArIUoJxkA', 'A9izmvFqfxE', 14),
 ('wUArIUoJxkA', 'g9jHl0VOQZA', 6),
 ('wUArIUoJxkA', 'DpsWZQIgqqU', 3),
 ('wUArIUoJxkA', 'TJ04gk-hoTc', 5),
 ('oHg8ETcoXIA', 'YydAr-kkobs', 43),
 ('oHg8ETcoXIA', 'xpU3TBmoZB4', 80),
 ('oHg8ETcoXIA', 'k7HHjV_IzPs', 5),
 ('oHg8ETcoXIA', '_E33gRrWUHM', 5),
 ('oHg8ETcoXIA', 'A9izmvFqfxE', 55),
 ('oHg8ETcoXIA', 'g9jHl0VOQZA', 34),
 ('oHg8ETcoXIA', 'DpsWZQIgqqU', 7),
 ('oHg8ETcoXIA', 'TJ04gk-hoTc', 9),
 ('YydAr-kkobs', 'xpU3TBmoZB4', 45),
 ('YydAr-kkobs', 'k7HHjV_IzPs', 3),
 ('YydAr-kkobs', '_E33gRrWUHM', 8),
 ('YydAr-kkobs', 'A9izmvFqfxE', 62),
 ('YydAr-kkobs', 'g9jHl0VOQZA', 25),
 ('YydAr-kkobs', 'DpsWZQIgqqU', 5),
 ('YydAr-kkobs', 'TJ04gk-hoTc', 9),
 ('xpU3TBmoZB4', 'k7HHjV_IzPs', 6),
 ('xpU3TBmoZB4', '_E33gRrWUHM', 4),
 ('xpU3TBmoZB4', 'A9izmvFqfxE', 84),
 ('xpU3TBmoZB4',